In [1]:
import os
from pathlib import Path

import pandas as pd
import plotly.express as px

In [4]:
class Config:
    data_dir = Path("../data")

In [8]:
customers = pd.read_csv(os.path.join(Config.data_dir, "customers.csv"))
noncustomers = pd.read_csv(os.path.join(Config.data_dir, "noncustomers.csv"))
usage = pd.read_csv(os.path.join(Config.data_dir, "usage_actions.csv"))

In [9]:
customers.head()

,CLOSEDATE,MRR,ALEXA_RANK,EMPLOYEE_RANGE,INDUSTRY,id
0,2019-06-20,290.00,309343,201 to 1000,Other,199
1,2019-03-22,392.54,16000001,51 to 200,NaN,147
2,2020-01-08,-61.15,60502,1001 to 10000,HIGHER_EDUCATION,118
3,2019-09-19,400.00,3575321,201 to 1000,CONSUMER_SERVICES,126
4,2019-01-27,209.98,273063,26 to 50,Technology - Software,174


In [10]:
noncustomers.head()

,ALEXA_RANK,EMPLOYEE_RANGE,INDUSTRY,id
0,4489573.0,11 to 25,NaN,4916
1,4027013.0,6 to 10,NaN,3489
2,16000001.0,26 to 50,COMPUTER_SOFTWARE,2813
3,814822.0,1001 to 10000,HOSPITAL_HEALTH_CARE,727
4,16000001.0,26 to 50,NaN,2166


In [11]:
usage.head()

,WHEN_TIMESTAMP,ACTIONS_CRM_CONTACTS,ACTIONS_CRM_COMPANIES,ACTIONS_CRM_DEALS,ACTIONS_EMAIL,USERS_CRM_CONTACTS,USERS_CRM_COMPANIES,USERS_CRM_DEALS,USERS_EMAIL,id
0,2020-06-01 00:00:00.000,278,0,34,0,2,0,2,0,616
1,2020-04-20 00:00:00.000,0,0,0,0,0,0,0,0,3059
2,2019-04-29 00:00:00.000,210,1,17,11,3,1,4,1,2813
3,2020-02-24 00:00:00.000,1,1,1,0,1,1,1,0,5076
4,2020-02-17 00:00:00.000,9,9,7,0,1,1,1,0,3983


In [14]:
print("The dataset contains:")
print(f"{customers.shape[0]} customers")
print(f"{noncustomers.shape[0]} non-customers")
print(f"{usage.shape[0]} usage actions")

The dataset contains:
200 customers
5003 non-customers
25387 usage actions


Lets assign categorical variables to customers and non-customers to differentiate labels

In [15]:
customers["is_customer"] = 1
noncustomers["is_customer"] = 0

base = pd.concat([customers, noncustomers], axis=0).reset_index(drop=True)

In [16]:
base.head()

,CLOSEDATE,MRR,ALEXA_RANK,EMPLOYEE_RANGE,INDUSTRY,id,is_customer
0,2019-06-20,290.00,309343.0,201 to 1000,Other,199,1
1,2019-03-22,392.54,16000001.0,51 to 200,NaN,147,1
2,2020-01-08,-61.15,60502.0,1001 to 10000,HIGHER_EDUCATION,118,1
3,2019-09-19,400.00,3575321.0,201 to 1000,CONSUMER_SERVICES,126,1
4,2019-01-27,209.98,273063.0,26 to 50,Technology - Software,174,1


In [20]:
base["is_customer"].value_counts(normalize=True)

is_customer
0    0.961561
1    0.038439
Name: proportion, dtype: float64

We have a highly imbalanced dataset with only a few in proportion as customers compared to non customers

Next up we want a single row per company for modeling. We will aggregate usage_actions by id.

In [23]:
usage_summary = (
    usage.groupby("id")
    .agg(
        {
            "ACTIONS_CRM_CONTACTS": "sum",
            "ACTIONS_CRM_COMPANIES": "sum",
            "ACTIONS_CRM_DEALS": "sum",
            "ACTIONS_EMAIL": "sum",
            "USERS_CRM_CONTACTS": "mean",
            "USERS_CRM_COMPANIES": "mean",
            "USERS_CRM_DEALS": "mean",
            "USERS_EMAIL": "mean",
        }
    )
    .reset_index()
)
usage_summary.head()

,id,ACTIONS_CRM_CONTACTS,ACTIONS_CRM_COMPANIES,ACTIONS_CRM_DEALS,ACTIONS_EMAIL,USERS_CRM_CONTACTS,USERS_CRM_COMPANIES,USERS_CRM_DEALS,USERS_EMAIL
0,1,14845,1889,5253,70,4.985714,3.214286,3.800000,0.285714
1,2,4970,798,790,247,1.931507,0.863014,1.013699,0.479452
2,3,43467,8219,18797,498,16.581081,10.635135,9.959459,1.459459
3,4,54329,77,867,0,8.000000,0.486111,2.666667,0.000000
4,5,5172,1450,3551,70,5.347222,2.444444,4.111111,0.375000


Now each company has summarized activity metrics (how much they used the CRM).

In [25]:
final_df = base.merge(usage_summary, on="id", how="left")

In [26]:
final_df.head()

,CLOSEDATE,MRR,ALEXA_RANK,EMPLOYEE_RANGE,INDUSTRY,id,is_customer,ACTIONS_CRM_CONTACTS,ACTIONS_CRM_COMPANIES,ACTIONS_CRM_DEALS,ACTIONS_EMAIL,USERS_CRM_CONTACTS,USERS_CRM_COMPANIES,USERS_CRM_DEALS,USERS_EMAIL
0,2019-06-20,290.00,309343.0,201 to 1000,Other,199,1,5438.0,24.0,125.0,2464.0,7.985714,0.314286,0.528571,7.528571
1,2019-03-22,392.54,16000001.0,51 to 200,NaN,147,1,6069.0,2935.0,3461.0,108.0,4.656250,4.250000,5.328125,0.671875
2,2020-01-08,-61.15,60502.0,1001 to 10000,HIGHER_EDUCATION,118,1,65199.0,8855.0,21252.0,742.0,14.558824,9.779412,10.720588,2.338235
3,2019-09-19,400.00,3575321.0,201 to 1000,CONSUMER_SERVICES,126,1,8574.0,728.0,64.0,94.0,2.442623,0.836066,0.360656,0.622951
4,2019-01-27,209.98,273063.0,26 to 50,Technology - Software,174,1,16964.0,5230.0,8360.0,372.0,8.898551,6.217391,4.797101,0.913043


In [28]:
final_df[1000:1005]

,CLOSEDATE,MRR,ALEXA_RANK,EMPLOYEE_RANGE,INDUSTRY,id,is_customer,ACTIONS_CRM_CONTACTS,ACTIONS_CRM_COMPANIES,ACTIONS_CRM_DEALS,ACTIONS_EMAIL,USERS_CRM_CONTACTS,USERS_CRM_COMPANIES,USERS_CRM_DEALS,USERS_EMAIL
1000,NaN,NaN,16000001.0,201 to 1000,NaN,907,0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1001,NaN,NaN,327211.0,201 to 1000,PRIMARY_SECONDARY_EDUCATION,4239,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,NaN,NaN,168347.0,201 to 1000,MARKETING_AND_ADVERTISING,340,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,NaN,NaN,16000001.0,6 to 10,NaN,4330,0,6.0,12.0,2.0,0.0,3.0,3.0,1.0,0.0
1004,NaN,NaN,NaN,1001 to 10000,NaN,4283,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Each row is now a company profile with:

Static features → ALEXA_RANK, EMPLOYEE_RANGE, INDUSTRY

Behavioral features → summed/average CRM usage

Target variable → label (converted or not)

In [29]:
final_df.shape, final_df.isna().mean().sort_values(ascending=False).head(10)

((5203, 15),
 CLOSEDATE                0.961561
 MRR                      0.961561
 INDUSTRY                 0.740727
 ACTIONS_CRM_CONTACTS     0.313857
 ACTIONS_CRM_COMPANIES    0.313857
 ACTIONS_CRM_DEALS        0.313857
 ACTIONS_EMAIL            0.313857
 USERS_CRM_CONTACTS       0.313857
 USERS_CRM_COMPANIES      0.313857
 USERS_CRM_DEALS          0.313857
 dtype: float64)

In [30]:
fig = px.bar(
    final_df["is_customer"].value_counts().rename_axis("is_customer").reset_index(name="counts"),
    x="is_customer",
    y="counts",
    text="counts",
    title="Distribution of Customers vs Non-Customers",
)
fig.update_traces(textposition="outside")
fig.show()

In [34]:
numeric_cols = [
    c for c in final_df.columns if pd.api.types.is_numeric_dtype(final_df[c]) and c not in ["id", "is_customer"]
]
for col in ["MRR", "ACTIONS_CRM_DEALS", "ACTIONS_EMAIL"]:
    if col in final_df.columns:
        fix = px.histogram(
            final_df,
            x=col,
            color="is_customer",
            barmode="overlay",
            nbins=50,
            opacity=0.6,
            marginal="box",
            title=f"Distribution of {col} by Customer Status",
        )
        fig.show()

In [35]:
def plot_conversion_rate(col):
    if col not in final_df.columns:
        return
    tmp = (
        final_df.groupby(col)["is_customer"]
        .mean()
        .reset_index()
        .rename(columns={"is_customer": "conversion_rate"})
        .sort_values("conversion_rate", ascending=False)
    )
    fig = px.bar(tmp, x=col, y="conversion_rate", title=f"Conversion rate by {col}", text="conversion_rate")
    fig.update_traces(texttemplate="%{text:.2%}", textposition="outside")
    fig.update_layout(yaxis_tickformat="%")
    fig.show()


for col in ["EMPLOYEE_RANGE", "INDUSTRY"]:
    plot_conversion_rate(col)

In [37]:
corr_cols = [c for c in numeric_cols if c in final_df.columns] + ["is_customer"]
corr = final_df[corr_cols].corr()

fig = px.imshow(corr, text_auto=".2f", aspect="auto", title="Correlation Matrix")
fig.show()